In [1]:
import matplotlib.pylab as plt
import numpy as np
from scipy.interpolate import griddata
import pandas as pd
from scipy import optimize
import math
from matplotlib.patches import Polygon

from RoutePlanner.Mesh import Mesh

In [2]:
IceExtent = {}
IceExtent['X']      = np.load('./resources/IceContent_X.npy') 
IceExtent['Y']      = np.load('./resources/IceContent_Y.npy') 
IceExtent['Values'] = np.load('./resources/IceContent_V.npy') 
IceExtent['Mask']   = np.load('./resources/IceContent_mask.npy') 
vls = IceExtent['Values']
vls[IceExtent['Mask']] = 1.0
IceExtent['Values'] = vls


Coastline = {}
Coastline['X']      = np.load('./resources/IceContent_X.npy') 
Coastline['Y']      = np.load('./resources/IceContent_Y.npy')
values = np.load('./resources/IceContent_V.npy'); values[values < 1] = 0
Coastline['Values'] = values

Currents = {}
Currents['X']  =  np.load('./resources/Current_X.npy') 
Currents['Y']  =  np.load('./resources/Current_Y.npy') 
Currents['Vx'] = np.load('./resources/Current_Vx.npy')
Currents['Vy'] = np.load('./resources/Current_Vy.npy')

In [3]:
MeshInfo = {}
MeshInfo['Xmin'] = 290
MeshInfo['Xmax'] = 310
MeshInfo['Ymin'] = -70
MeshInfo['Ymax'] = -60
MeshInfo['dx']   = 5
MeshInfo['dy']   = 2.5

MeshInfo['CoastMask'] = {}
MeshInfo['CoastMask']['X']          = Coastline['X'] 
MeshInfo['CoastMask']['Y']          = Coastline['Y'] 
MeshInfo['CoastMask']['Mask']       = Coastline['Values'].astype(bool)
MeshInfo['CoastMask']['Split']      = True
MeshInfo['CoastMask']['SplitDepth'] = 3

MeshInfo['IceExtent'] = {}
MeshInfo['IceExtent']['X']          = IceExtent['X'] 
MeshInfo['IceExtent']['Y']          = IceExtent['Y'] 
MeshInfo['IceExtent']['Values']     = IceExtent['Values']
MeshInfo['IceExtent']['Mask']       = IceExtent['Mask']
MeshInfo['IceExtent']['Split']      = True
MeshInfo['IceExtent']['SplitDepth'] = 3
MeshInfo['IceExtent']['SplitDiff']  = 0.1
MeshInfo['IceExtent']['MaxProportion'] = 0.8

MeshInfo['Currents'] = {}
MeshInfo['Currents']['X']  = Currents['X'] 
MeshInfo['Currents']['Y']  = Currents['Y']
MeshInfo['Currents']['Vx'] = Currents['Vx'] 
MeshInfo['Currents']['Vy'] = Currents['Vy'] 




### Generating Mesh from Land & Ice Content

In [4]:
mesh = Mesh(MeshInfo)
mesh.IceInformation()
mesh.VectorInformation()


=============== Initialising Mesh ================
Xmin=290,Xmax=310,dX=5  Ymin=-70,Ymax=-60,dY=2.5

============ GRIDDING LAND INFORMATION ===========
~~~~~~~~ Initialising Original Mask Array ~~~~~~~~
~~~~~~~~ Splitting Mask array based on Coast ~~~~~~~~
-----> Max Depth=3

============ GRIDDING ICE INFORMATION ===========
~~~~~~~~ Initialising Original Mask Array ~~~~~~~~
~~~~~~~~  Splitting Mask array based on Ice Content ~~~~~~~~
-----> Max Depth=3

============ GRIDDING Current Data ===============


c:\Users\jonsmi\OneDrive - NERC\Documents\BAS_RoutePlanner\SDADT-pyRoutePlanner\RoutePlanner\Mesh.py:139: RuntimeWarning: Mean of empty slice
  self.cells[cc].vector = np.array([np.nanmean(gVx),np.nanmean(gVy)])


### Plotting Generated Mesh

In [5]:
PlotMesh(mesh)

NameError: name 'PlotMesh' is not defined

# Optimisation

In [ ]:
OptInfo = {}
OptInfo['WayPoints']            = pd.read_csv('./resources/WayPoints_SmallRegion.csv') 
OptInfo['VehicleInfo']          = {}
OptInfo['VehicleInfo']['Speed'] = 26.3*(1000/(60*60))

In [ ]:
TT = TravelTime(mesh,OptInfo)
TT.Dijkstra(verbrose=True)

In [ ]:
OptimisedPaths(mesh,TT,TT.Paths)

In [ ]:
TT.PathSmoothing()
OptimisedPaths(mesh,TT,TT.Paths)